<a href="https://colab.research.google.com/github/jellyXuuuuu/CovidNetDeepLearning/blob/main/train_covid_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare for package

# upload date from drive(temporarily)

In [ ]:
# !cp 'drive/MyDrive/data.zip' .

In [ ]:
# !unzip -d data/ ./data.zip

# load data from kaggle

In [ ]:
!pip install kaggle

In [4]:
# !export PATH="/home/user1/.local/bin:$PATH"

In [16]:
# !pip install kaggle
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json
api_token = {"username":"yyux1108","key":"2bb1c1cbae6df563c430a39c690da7a3"}


mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [17]:
import json

with open('/usr/local/bin/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

In [18]:
!kaggle datasets download -d andyczhao/covidx-cxr2

100% 12.9G/12.9G [01:51<00:00, 177MB/s]
100% 12.9G/12.9G [01:51<00:00, 124MB/s]


In [19]:
!unzip -d data/ ./covidx-cxr2.zip

Streaming output truncated to the last 5000 lines.
  inflating: data/train/aead3682-bdf4-4f43-8468-8e6ac3cd41fa.png  
  inflating: data/train/aeae6cbc-d713-4760-845c-fcc93ed5ad64.png  
  inflating: data/train/aeaf6ffd-ff7e-48cc-b31f-175a5e79b415.png  
  inflating: data/train/aeb2a32c-595a-44c2-b827-c6ce0f6ff4f3.png  
  inflating: data/train/aeb6324d-08d5-426b-9a60-bc39d5151edb.png  
  inflating: data/train/aeb99fe0-d9f2-4122-8abf-e7f933f71f80.png  
  inflating: data/train/aebb48db-2f6f-41ad-87e7-d90c5620187e.png  
  inflating: data/train/aebb5625-cbea-4e11-9a9a-19708903ed1b.png  
  inflating: data/train/aebf9717-f500-44ac-87cd-273f24e0c3ce.png  
  inflating: data/train/aec1daa4-1910-4270-8de2-9abe6f3b1bf0.png  
  inflating: data/train/aec5b2e5-ceff-4e21-a089-19efcd0c3d45.png  
  inflating: data/train/aec849c0-4ef4-4d4e-9d09-573f1ce4cbd1.png  
  inflating: data/train/aec8c4ac-c66d-4523-a3df-278d0b7bead7.png  
  inflating: data/train/aeca7c41-8768-4f70-8bc7-5427c4718782.png  
  inflating

# Actual experiment

In [20]:
!cp 'drive/MyDrive/covid/eval.py' .

In [21]:
!cp 'drive/MyDrive/covid/data.py' .

In [23]:
!cp -r 'drive/MyDrive/covid/models/' .

In [ ]:
from __future__ import print_function
import tensorflow as tf
import os, argparse, pathlib

from eval import eval
from data import BalanceCovidDataset

# To remove TF Warnings
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

parser = argparse.ArgumentParser(description='COVID-Net Training Script')
parser.add_argument('--epochs', default=100, type=int, help='Number of epochs')
parser.add_argument('--lr', default=0.0002, type=float, help='Learning rate')
parser.add_argument('--bs', default=8, type=int, help='Batch size')
parser.add_argument('--weightspath', default='models/COVIDNet-CXR-2', type=str,
                    help='Path to model files, defaults to \'models/COVIDNet-CXR-2\'')
parser.add_argument('--metaname', default='model.meta', type=str, help='Name of ckpt meta file')
parser.add_argument('--ckptname', default='model', type=str, help='Name of model ckpts')
parser.add_argument('--n_classes', default=2, type=int, help='Number of detected classes, defaults to 2')
parser.add_argument('--trainfile', default='labels/train_COVIDx9B.txt', type=str, help='Path to train file')
parser.add_argument('--testfile', default='labels/test_COVIDx9B.txt', type=str, help='Path to test file')
parser.add_argument('--name', default='COVIDNet', type=str, help='Name of folder to store training checkpoints')
parser.add_argument('--datadir', default='data', type=str, help='Path to data folder')
parser.add_argument('--covid_weight', default=1., type=float, help='Class weighting for covid')
parser.add_argument('--covid_percent', default=0.5, type=float, help='Percentage of covid samples in batch')
parser.add_argument('--input_size', default=480, type=int, help='Size of input (ex: if 480x480, --input_size 480)')
parser.add_argument('--top_percent', default=0.08, type=float, help='Percent top crop from top of image')
parser.add_argument('--in_tensorname', default='input_1:0', type=str, help='Name of input tensor to graph')
parser.add_argument('--out_tensorname', default='norm_dense_2/Softmax:0', type=str,
                    help='Name of output tensor from graph')
parser.add_argument('--logit_tensorname', default='norm_dense_2/MatMul:0', type=str,
                    help='Name of logit tensor for loss')
parser.add_argument('--label_tensorname', default='norm_dense_1_target:0', type=str,
                    help='Name of label tensor for loss')
parser.add_argument('--weights_tensorname', default='norm_dense_1_sample_weights:0', type=str,
                    help='Name of sample weights tensor for loss')
parser.add_argument('--training_tensorname', default='keras_learning_phase:0', type=str,
                    help='Name of training placeholder tensor')
parser.add_argument('--is_severity_model', action='store_true', 
                    help='Add flag if training COVIDNet CXR-S model')

args = parser.parse_args(args = [])
'''
'python train_tf.py \
    --weightspath models/COVIDNet-CXR-2 \
    --metaname model.meta \
    --ckptname model \
    --n_classes 2 \
    --trainfile labels/train_COVIDx9B.txt \
    --testfile labels/test_COVIDx9B.txt \
    --out_tensorname norm_dense_2/Softmax:0 \
    --logit_tensorname norm_dense_2/MatMul:0'
'''

args.weightspath = 'models/COVIDNet-CXR-2' 
args.metaname = 'model.meta'
args.ckptname = 'model'
args.n_classes = 2
args.trainfile = 'data/train.txt'
args.testfile = 'data/test.txt'
args.out_tensorname = 'norm_dense_2/Softmax:0'
args.logit_tensorname = 'norm_dense_2/MatMul:0'

# ///////////////// IMPORTANT!!!! Missed in given command \\\\\\\\\\\\\\\\\\\\\\\\\\
args.datadir = './data'

# Parameters
learning_rate = args.lr
batch_size = args.bs
display_step = 1

# output path
outputPath = './output/'
runID = args.name + '-lr' + str(learning_rate)
runPath = outputPath + runID
pathlib.Path(runPath).mkdir(parents=True, exist_ok=True)
print('Output: ' + runPath)

with open(args.trainfile) as f:
    trainfiles = f.readlines()
with open(args.testfile) as f:
    testfiles = f.readlines()

if args.is_severity_model:
    # For COVIDNet CXR-S severity level 1 and 2 detection using COVIDxSev dataset
    mapping = {
        'level2': 0,
        'level1': 1 
    }
    # For COVIDxSev use a 50/50 balanced batch with 1:1 sample weights
    class_weights = [1., 1.]
    args.covid_percent = 0.5  
elif args.n_classes == 2:
    # For COVID-19 positive/negative detection
    mapping = {
        'negative': 0,
        'positive': 1,
    }
    class_weights = [1., args.covid_weight]
elif args.n_classes == 3:
    # For detection of no pneumonia/non-COVID-19 pneumonia/COVID-19 pneumonia
    mapping = {
        'normal': 0,
        'pneumonia': 1,
        'COVID-19': 2
    }
    class_weights = [1., 1., args.covid_weight]
else:
    raise Exception('''COVID-Net currently only supports 2 class COVID-19 positive/negative detection
        or 3 class detection of no pneumonia/non-COVID-19 pneumonia/COVID-19 pneumonia''')

generator = BalanceCovidDataset(data_dir=args.datadir,
                                csv_file=args.trainfile,
                                batch_size=batch_size,
                                input_shape=(args.input_size, args.input_size),
                                n_classes=args.n_classes,
                                mapping=mapping,
                                covid_percent=args.covid_percent,
                                class_weights=class_weights,
                                top_percent=args.top_percent,
                                is_severity_model=args.is_severity_model)

with tf.compat.v1.Session() as sess:
    tf.compat.v1.get_default_graph()
    saver = tf.compat.v1.train.import_meta_graph(os.path.join(args.weightspath, args.metaname))
    print("test for weightspath", os.path.join(args.weightspath, args.metaname))


    graph = tf.compat.v1.get_default_graph()

    image_tensor = graph.get_tensor_by_name(args.in_tensorname)
    labels_tensor = graph.get_tensor_by_name(args.label_tensorname)
    sample_weights = graph.get_tensor_by_name(args.weights_tensorname)
    pred_tensor = graph.get_tensor_by_name(args.logit_tensorname)
    training_tensor = graph.get_tensor_by_name(args.training_tensorname)
    # loss expects unscaled logits since it performs a softmax on logits internally for efficiency

    # Define loss and optimizer
    loss_op = tf.compat.v1.reduce_mean(tf.compat.v1.nn.softmax_cross_entropy_with_logits_v2(
        logits=pred_tensor, labels=labels_tensor)*sample_weights)
    optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op)

    # Initialize the variables
    init = tf.compat.v1.global_variables_initializer()

    # Run the initializer
    sess.run(init)

    # load weights
    saver.restore(sess, os.path.join(args.weightspath, args.ckptname))
    #saver.restore(sess, tf.train.latest_checkpoint(args.weightspath))

    # save base model
    saver.save(sess, os.path.join(runPath, 'model'))
    print('Saved baseline checkpoint')
    print('Baseline eval:')
    print("datadir", args.datadir)
    eval(sess, graph, testfiles, os.path.join(args.datadir,'test'),
         args.in_tensorname, args.out_tensorname, args.input_size, mapping)

    # Training cycle
    print('Training started')
    total_batch = len(generator)
    progbar = tf.keras.utils.Progbar(total_batch)
    for epoch in range(args.epochs):
        for i in range(total_batch):
            # Run optimization
            batch_x, batch_y, weights, is_training = next(generator)
            sess.run(train_op, feed_dict={image_tensor: batch_x,
                                          labels_tensor: batch_y,
                                          sample_weights: weights,
                                          training_tensor: is_training})
            progbar.update(i+1)

        if epoch % display_step == 0:
            pred = sess.run(pred_tensor, feed_dict={image_tensor:batch_x})
            loss = sess.run(loss_op, feed_dict={pred_tensor: pred,
                                                labels_tensor: batch_y,
                                                sample_weights: weights})
            print("Epoch:", '%04d' % (epoch + 1), "Minibatch loss=", "{:.9f}".format(loss))
            eval(sess, graph, testfiles, os.path.join(args.datadir,'test'),
                 args.in_tensorname, args.out_tensorname, args.input_size, mapping)
            saver.save(sess, os.path.join(runPath, 'model'), global_step=epoch+1, write_meta_graph=False)
            print('Saving checkpoint at epoch {}'.format(epoch + 1))


print("Optimization Finished!")


Output: ./output/COVIDNet-lr0.0002
13992 15994
test for weightspath models/COVIDNet-CXR-2/model.meta
Saved baseline checkpoint
Baseline eval:
datadir ./data
